<a href="https://colab.research.google.com/github/dantae74/machine-learning/blob/main/13-rnn-lotto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

모두를 위한 머신러닝에서 가져왔습니다.
# RNN-lotto

In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
url = 'https://raw.githubusercontent.com/dantae74/machine-learning/main/dataset/lotto.csv'
df = pd.read_csv(url)

In [3]:
df.head()

,seq,date,num,won,1,2,3,4,5,6
0,964,2021.5.22,10,"2,345,861,063",6,21,36,38,39,43
1,963,2021.5.15,15,"1,476,478,125",6,12,19,23,34,42
2,962,2021.5.8,12,"1,940,906,094",1,18,28,31,34,43
3,961,2021.5.1,9,"2,575,231,209",11,20,29,31,33,42
4,960,2021.4.24,10,"2,401,133,213",2,18,24,30,32,45


In [4]:
df = df.sort_values('seq', ascending=True)

In [5]:
df.head()

,seq,date,num,won,1,2,3,4,5,6
963,1,2002.12.7,0,0,10,23,29,33,37,40
962,2,2002.12.14,1,"2,002,006,800",9,13,21,25,32,42
961,3,2002.12.21,1,"2,000,000,000",11,16,19,21,27,31
960,4,2002.12.28,0,0,14,27,30,31,40,42
959,5,2003.1.4,0,0,16,24,29,40,41,42


In [6]:
df.shape

(964, 10)

In [7]:
ncount = np.zeros((1,45), dtype=float)

In [8]:
for i in range(df.shape[0]):
  for j in range(6):
    col = df.iloc[i][str(j+1)]
    ncount[0][col-1] += 1.0

  if i == 0:
    X = ncount
  else:
    X = np.append(X, ncount, axis=0)

print('x.shape:', X.shape)
# X[964]

x.shape: (964, 45)


In [9]:
X[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 1., 0., 0., 0., 0.,
       0., 0., 0., 1., 0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 1., 1., 0.,
       0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0.])

In [10]:
df_y = df[['1','2','3','4','5','6']]

In [11]:
Y = df_y.values

In [12]:
print('x.shape:', X.shape)
print('y.shape:', Y.shape)

x.shape: (964, 45)
y.shape: (964, 6)


In [13]:
y_onehot = []
tmponehot = np.zeros(45).tolist()
tmponehot[3] = 1.0
y_onehot.append(tmponehot)
y_onehot.append(tmponehot)

np.array(y_onehot).shape


(2, 45)

In [14]:
def make_onehot(value):
  onehot = np.zeros(45).tolist()
  onehot[value-1] = 1.0
  return onehot

In [15]:
y_onehot = []
for i in range(Y.shape[0]):
  for j in range(6):
    y_onehot.append(make_onehot(Y[i][j]))

y_onehot = np.array(y_onehot)
y_onehot = np.reshape(y_onehot, (964,6,45))
print('y_onehot.shape:', y_onehot.shape)


y_onehot.shape: (964, 6, 45)


In [16]:
X.shape

(964, 45)

In [17]:
# build datasets
def build_dataset(x_time_series, y_time_series, seq_length):
    dataX = []
    dataY = []
    for i in range(0, len(y_time_series) - seq_length):
        x = x_time_series[i:i + seq_length, :]
        y = y_time_series[i + seq_length - 1, :]
        dataX.append(x)
        dataY.append(y)
    return np.array(dataX), np.array(dataY)

In [18]:
x_data, y_data = build_dataset(X, y_onehot, 6)

In [19]:
print('x_data.shape:', x_data.shape)
print('y_data.shape:', y_data.shape)

x_data.shape: (958, 6, 45)
y_data.shape: (958, 6, 45)


In [20]:
x_data[0]
y_data[0]

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 

In [21]:
model = keras.Sequential();
model.add(keras.layers.LSTM(units=45, input_shape=(6, 45), return_sequences=True))
model.add(keras.layers.LSTM(units=45, return_sequences=True))
model.add(keras.layers.Dense(units=45, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(learning_rate=0.01), metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 6, 45)             16380     
_________________________________________________________________
lstm_1 (LSTM)                (None, 6, 45)             16380     
_________________________________________________________________
dense (Dense)                (None, 6, 45)             2070      
Total params: 34,830
Trainable params: 34,830
Non-trainable params: 0
_________________________________________________________________


In [22]:
model.fit(x_data, y_data, epochs=1000, verbose=0)

In [23]:
x_test = x_data[-1,:]
x_test = np.delete(x_test, 0, axis=0)
x_test = np.append(x_test, X[-1:], axis=0)
x_test = x_test.reshape((1,6,45))
print('x_test.shape:', x_test.shape)

x_test.shape: (1, 6, 45)


In [24]:
pred_y = model.predict(x_test)

In [25]:
for i in range(6):
  print(np.argmax(pred_y[0][i]) + 1)

1
8
20
31
34
45
